### Map Area

[San Francisco](https://goo.gl/PWSfft), in northern California, is a hilly city on the tip of a peninsula surrounded by the Pacific Ocean and San Francisco Bay. It's known for its year-round fog, iconic Golden Gate Bridge, cable cars and colorful Victorian houses. The Financial District's Transamerica Pyramid is its most distinctive skyscraper. In the bay sits Alcatraz Island, site of the notorious former prison. 

I choose San Francisco as the area for this project because I was planning a trip to this city, so it would be a great chance for me to explore the city while I prepare for my visit.

Link to San Francisco metro extract:
https://mapzen.com/data/metro-extracts/metro/san-francisco_california/

### Load File & Import Functions

In [1]:
OSMFILE = 'san-francisco_california.osm'
import functions_osm_auditing as auditing
import functions_osm_database as database

### Improving Street Names

One major problem in the Map is that the street names are not uniformalized - for example some street names are overabbreviated. 
So what I am going to do here is to get the street types of loaded osm file, and update mapping dictionary accordingly.

In [2]:
# define mapping dictionary
mapping = { "St": "Street",
            "St.": "Street",
            "Rd.": "Road",
            "Ave": "Avenue",
            "Rd" : "Road"
            }

# get the street types of loaded osm file
street_types_found = auditing.audit(OSMFILE)

# add newly-found street types in osm file into pre-defined mapping above
new_mapping = auditing.update_mapping(mapping, street_types_found)

### Preparing for Database

#### Writing into XML
Rename street names found in osm file according to the new_mapping dictionary that I just updated, and save the processed street names into a new xml file.

In [3]:
auditing.clean_osm(OSMFILE,new_mapping,'processed_SanFrancisco')

#### Writing into csv
Save processed xml file into 5 csv files for later database build.

In [4]:
# specify path for later SQL database build usage
OSM_PATH = "processed_SanFrancisco.xml"
NODES_PATH = "SanFrancisco_nodes.csv"
NODE_TAGS_PATH = "SanFrancisco_nodes_tags.csv"
WAYS_PATH = "SanFrancisco_ways.csv"
WAY_NODES_PATH = "SanFrancisco_ways_nodes.csv"
WAY_TAGS_PATH = "SanFrancisco_ways_tags.csv"

# save processed xml file into csv files
database.process_map(OSM_PATH, False)

### Load in SQL

Import packages for SQL and for query result printout

In [5]:
import sqlite3
import pprint

# connect to database
db = sqlite3.connect('SanFrancisco.db')
db.text_factory = str
cur = db.cursor()

Create SQL tables for later query

In [6]:
# create table 'nodes'
import csv

cur.execute('''CREATE TABLE nodes(
    id INTEGER PRIMARY KEY NOT NULL,
    lat REAL,
    lon REAL,
    user TEXT,
    uid INTEGER,
    version INTEGER,
    changeset INTEGER,
    timestamp TEXT
);''')


with open(NODES_PATH,'r') as nodes:
    dr = csv.DictReader(nodes)
    to_db = [(i['id'], i['lat'], i['lon'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO nodes VALUES (?,?,?,?,?,?,?,?);", to_db)

db.commit()

In [7]:
# create table 'nodes_tags'

cur.execute('''CREATE TABLE nodes_tags (
    id INTEGER,
    key TEXT,
    value TEXT,
    type TEXT,
    FOREIGN KEY (id) REFERENCES nodes(id)
);
''')


with open(NODE_TAGS_PATH,'r') as nodes_tags:
    dr = csv.DictReader(nodes_tags)
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO nodes_tags VALUES (?,?,?,?);", to_db)

db.commit()

In [8]:
# create table 'ways'

cur.execute('''CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);
''')


with open(WAYS_PATH,'r') as ways:
    dr = csv.DictReader(ways)
    to_db = [(i['id'], i['user'], i['uid'], i['version'], i['changeset'], i['timestamp']) for i in dr]

cur.executemany("INSERT INTO ways VALUES (?,?,?,?,?,?);", to_db)

db.commit()

In [9]:
# create table 'ways_nodes'

cur.execute('''CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);
''')


with open(WAY_NODES_PATH,'r') as ways_nodes:
    dr = csv.DictReader(ways_nodes)
    to_db = [(i['id'], i['node_id'], i['position']) for i in dr]

cur.executemany("INSERT INTO ways_nodes VALUES (?,?,?);", to_db)

db.commit()

In [10]:
# create table 'ways_tags'

cur.execute('''CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);
''')


with open(WAY_TAGS_PATH,'r') as ways_tags:
    dr = csv.DictReader(ways_tags)
    to_db = [(i['id'], i['key'], i['value'], i['type']) for i in dr]

cur.executemany("INSERT INTO ways_tags VALUES (?,?,?,?);", to_db)

db.commit()

### Data Exploration

#### File Sizes

In [11]:
import os
import math

def convert_size(size_bytes):
   if size_bytes == 0:
       return "0B"
   size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
   i = int(math.floor(math.log(size_bytes, 1024)))
   p = math.pow(1024, i)
   s = round(size_bytes / p, 2)
   return "%s %s" % (s, size_name[i])

print 'Size of original OSM file is around %s'%convert_size(os.path.getsize(OSMFILE))
print 'Size of processed csv file, nodes, is around %s'%convert_size(os.path.getsize(NODES_PATH))
print 'Size of processed csv file, nodes_tags, is around %s'%convert_size(os.path.getsize(NODE_TAGS_PATH))
print 'Size of processed csv file, ways, is around %s'%convert_size(os.path.getsize(WAYS_PATH))
print 'Size of processed csv file, ways_nodes, is around %s'%convert_size(os.path.getsize(WAY_NODES_PATH))
print 'Size of processed csv file, ways_tags, is around %s'%convert_size(os.path.getsize(WAY_TAGS_PATH))

Size of original OSM file is around 1.31 GB
Size of processed csv file, nodes, is around 528.66 MB
Size of processed csv file, nodes_tags, is around 9.14 MB
Size of processed csv file, ways, is around 47.98 MB
Size of processed csv file, ways_nodes, is around 179.89 MB
Size of processed csv file, ways_tags, is around 56.27 MB


#### Contributing Users 

In [12]:
cur.execute("SELECT COUNT(DISTINCT(un.uid)) FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) un;")
print 'Number of unique users contribuing to this area of OpenStreetMap:'
pprint.pprint(cur.fetchall())

print '--'

cur.execute("SELECT un.user, COUNT(*) as total FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) un \
GROUP BY un.user ORDER BY total DESC LIMIT 5;")
print 'Top 5 users who contributed to the map area:'
pprint.pprint(cur.fetchall())

Number of unique users contribuing to this area of OpenStreetMap:
[(2750,)]
--
Top 5 users who contributed to the map area:
[('andygol', 1496526),
 ('ediyes', 887715),
 ('Luis36995', 679823),
 ('dannykath', 546182),
 ('RichRico', 415950)]


#### Number of Nodes & Ways

In [13]:
cur.execute("SELECT count(DISTINCT(id)) FROM nodes;")
print '# of nodes:'
pprint.pprint(cur.fetchall())

print '--'

cur.execute("SELECT count(DISTINCT(id)) FROM ways;")
print '# of ways:'
pprint.pprint(cur.fetchall())

# of nodes:
[(6609426,)]
--
# of ways:
[(822370,)]


#### But First, Coffee
See how many coffee shops are there in San Francisco for Blue Bottle, Philz, Starbucks and Peet's respectively

In [14]:
# See how many famouse 'Blue Bottle' coffee shops are there in San Francisco
cur.execute("SELECT count(*) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) un \
            WHERE un.value like '%Blue Bottle%' AND un.key = 'name';")
print '# of Blue Bottle Coffee:'
pprint.pprint(cur.fetchall())
print '--'

# How many 'Philz' are there? Their mint mojito is one of my favorite
cur.execute("SELECT count(*) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) un \
            WHERE un.value like '%Philz%' AND un.key = 'name';")
print '# of Philz Coffee:'
pprint.pprint(cur.fetchall())
print '--'

# How about 'Starbucks'
cur.execute("SELECT count(*) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) un \
            WHERE un.value like '%Starbucks' AND un.key = 'name';")
print '# of Starbucks Coffee:'
pprint.pprint(cur.fetchall())
print '--'

# How about "Peet's Coffee & Tea"?
cur.execute("SELECT count(*) FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) un \
            WHERE un.value like '%Peet%' AND un.key = 'name';")
print "# of Peet's Coffee & Tea:"
pprint.pprint(cur.fetchall())

# of Blue Bottle Coffee:
[(11,)]
--
# of Philz Coffee:
[(12,)]
--
# of Starbucks Coffee:
[(137,)]
--
# of Peet's Coffee & Tea:
[(64,)]


#### Going Spiritual
See how many data points are there for different religions respectively

In [15]:
cur.execute ("SELECT un.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) un \
            WHERE un.key = 'religion' GROUP BY un.value ORDER BY count DESC;")
pprint.pprint(cur.fetchall())

[('christian', 1062),
 ('buddhist', 43),
 ('jewish', 20),
 ('muslim', 10),
 ('unitarian_universalist', 5),
 ('hindu', 3),
 ('taoist', 3),
 ('perfect_liberty', 2),
 ('scientologist', 2),
 ('bahai', 1),
 ('eckankar', 1),
 ('humanist', 1),
 ('shinto', 1),
 ('spiritualist', 1)]


#### International Cuisines
San Francisco is a city with many immigrants and ethnic groups. I am curious what are the top cuisine types in this city

In [16]:
cur.execute ("SELECT un.value, COUNT(*) as count FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) un \
            WHERE un.key = 'cuisine' GROUP BY un.value ORDER BY count DESC LIMIT 20;")
pprint.pprint(cur.fetchall())

[('mexican', 300),
 ('coffee_shop', 267),
 ('pizza', 224),
 ('chinese', 207),
 ('burger', 200),
 ('japanese', 165),
 ('italian', 150),
 ('sandwich', 147),
 ('american', 139),
 ('thai', 117),
 ('vietnamese', 84),
 ('indian', 76),
 ('sushi', 75),
 ('asian', 60),
 ('seafood', 47),
 ('ice_cream', 46),
 ('french', 39),
 ('mediterranean', 26),
 ('regional', 25),
 ('chicken', 23)]


#### Museum Lover's Guide
A list of museums in San Francisco for a museum lover like me 

In [17]:
cur.execute ("SELECT distinct(nt.value) FROM nodes_tags nt JOIN \
            (SELECT distinct(id) FROM nodes_tags WHERE value='museum') un ON nt.id = un.id WHERE nt.key = 'name' \
            ORDER BY nt.value;")
pprint.pprint(cur.fetchall())

[('African American Museum and Library at Oakland',),
 ('Alameda Museum',),
 ('American Carousel Museum',),
 ('Ardenwood Forge',),
 ('Bay Area Discovery Museum',),
 ('Berkeley Historical Society',),
 ('Blake Garden',),
 ('Bolinas Museum',),
 ('California Crafts Museum',),
 ('California Historical Radio Society Bay Area Radio Museum',),
 ('California Historical Society Museum',),
 ('California Historical Society Museum & Bookstore',),
 ('California Society of Pioneers Museum and Library',),
 ('Cartoon Art Museum (closed)',),
 ('Childrens Multicultural Museum',),
 ('Chinese Historical Society of America',),
 ('Cohen Bray House',),
 ('Crissy Field Center',),
 ('CuriOdyssey',),
 ('Ebony Museum Of Art',),
 ('Exploratorium',),
 ('GLBT History Museum',),
 ('Golden State Model Railroad Museum',),
 ("Habitot Children's Museum",),
 ('Hearst Museum of Anthropology',),
 ('Jewish Community Museum',),
 ('Lightship Relief',),
 ('McConaghy House',),
 ('Morrison Planetarium',),
 ('Museo Italo Americano

In [18]:
db.close()

### Additional Ideas 

#### Challenge Encountered 
During wrangling with San Francisco's open street map, I encountered some issues that I found quite challenging:
- Size of file is quite big to process
- Many of street names are not standardized, e.g. street vs. str.
- Some street names are not at the end of string, which make data cleaning work more complicated. e.g. 'Sansome St #3500’


#### Improvement Suggestion

While exploring the dataset, there's no clue to me how a certain data point is rated. For example if I want to find a museum to visit, I have no idea how to select in the 50+ museums. It will be great to add additional information of how a place is rated.


#### Benefits/Problem with Improvement Discussed


Although adding rating information for datapoints, say rating of restaurants, would improve convenience to users, it will increase the complexity of maintaining data - since ratings change more often than other information, maintaining it for a total-volunteer-maintained map could be not that feasible. 


### References

- [Udacity DAND forum - Data Wrangling](https://discussions.udacity.com/c/nd002-data-wrangling)
- [Stackoverflow](https://stackoverflow.com)
- [csv Package Documentation](https://docs.python.org/2/library/csv.html)
- [SQLite Documentation](https://sqlite.org/docs.html)
- [xml.etree.ElementTree Documentation](https://docs.python.org/2/library/xml.etree.elementtree.html)
- [Convert File Sizes in Python](https://stackoverflow.com/questions/5194057/better-way-to-convert-file-sizes-in-python)
- [SQLite Module in Python](http://www.bogotobogo.com/python/python_sqlite_connect_create_drop_table.php)